In [3]:


import numpy as np

import tensorflow as tf


In [4]:
import os
import json
from zipfile import ZipFile
from PIL import Image

from tensorflow.keras import layers, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For working with the dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Utility
import cv2  # If additional image processing is needed
from pathlib import Path  # For easier file path management

In [5]:
!pip install kaggle


In [6]:
kaggle_credentails = json.load(open("/content/kaggle.json"))


In [7]:
# setup Kaggle API key as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_credentails["username"]
os.environ['KAGGLE_KEY'] = kaggle_credentails["key"]

In [8]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset


Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
plantvillage-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!ls


 kaggle.json		    sample_data			 test_potato_early_blight.jpg
'plantvillage dataset'	    test_apple_black_rot.JPG
 plantvillage-dataset.zip   test_blueberry_healthy.jpg


In [ ]:
with ZipFile("plantvillage-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
image = mpimg.imread("/content/plantvillage dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG")
# Convert the image from BGR (OpenCV default) to RGB (matplotlib default)
print(image)
# Display the image
print(image.shape)
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')  # Remove axis for better visualization
plt.title("Sample Image")
plt.show()

In [ ]:
batch_size=100
img_size = 224

# Image Data Generators
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)
# Dataset Path
base_dir = '/content/plantvillage dataset/color'

In [ ]:
# Train Generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

# Validation Generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

In [ ]:

tf.keras.optimizers.Adam(
    learning_rate=0.2)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(img_size, img_size, 3)),  # Explicitly define input shape
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
    ])



model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',       # Monitor validation loss
    patience=3,               # Stop after 3 epochs with no improvement
    mode='min',               # Minimize validation loss
    restore_best_weights=True # Restore weights from the best epoch
)

# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size  ,
    callbacks=[early_stopping]
)


In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))
# Model Evaluation


In [ ]:

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
img = Image.open("/content/test_apple_black_rot.JPG")
# Resize the image
img = img.resize((224, 224))
# Convert the image to a numpy array
img_array = np.array(img)
# Add batch dimension
img_array = np.expand_dims(img_array, axis=0)
# Scale the image values to [0, 1]
img_array = img_array.astype('float32') / 255.

In [ ]:
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]
class_indices = {v: k for k, v in train_generator.class_indices.items()}

predicted_class_name = class_indices[predicted_class_index]

In [ ]:

# Output the result
print("Predicted Class Name:", predicted_class_name)

In [ ]:
model.save('plant_disease_model.h5')


In [ ]:
!sudo pip3 install tensorflowjs


In [ ]:
!sudo tensorflowjs_converter


In [ ]:
!tensorflowjs_converter --input_format=keras '/content/plant_disease_prediction_model.h5' "/content/drive/MyDrive/json"

In [ ]:
!zip -r model.zip model


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/content/plant_disease_model.h5')

# Print model summary to verify input shape
model.summary()
